In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with importing a custom model evaluation to the Vertex AI Model Registry

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_evaluation%2Fget_started_with_custom_model_evaluation_import.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

**_NOTE_**: This notebook is tested in the following environment:

* Python version = 3.9

## Overview

This tutorial shows how to use Vertex AI Model Evaluation to import a custom model evaluation to an existing Vertex AI Model Registry entry.

Learn more about [Model evaluation in Vertex AI](https://cloud.google.com/vertex-ai/docs/evaluation/introduction).

### Objective

In this tutorial, you learn how to construct and upload a custom model evaluation, and upload the custom model evaluation to a model resource entry in Vertex AI Model Registry.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model Evaluation
- Vertex AI Model Registry

The steps performed include:

- Import a pretrained (blessed) model to the Vertex AI Model Registry.
- Construct a custom model evaluation.
- Import the model evaluation metrics to the corresponding model in the Vertex AI Model Registry.
- List the model evaluation for the corresponding model in the Vertex AI Model Registry.
- Construct a second custom model evaluation.
- Import the second model evaluation metrics to the corresponding model in the Vertex AI Model Registry.
- List the second model evaluation for the corresponding model in the Vertex AI Model Registry.

Learn more about [Model Evaluation in Vertex AI](https://cloud.google.com/vertex-ai/docs/evaluation/introduction).

### Model

This tutorial uses a pre-trained image classification model from TensorFlow Hub, which is trained on ImageNet dataset.

Learn more about [ResNet V2 pretained model](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5). 

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade google-cloud-aiplatform \
                                tensorflow==2.15.1 \
                                tensorflow-hub

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for prediction.

Set the variables `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa T4 GPUs allocated to each VM, you would specify:

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more about [hardware accelerator support for your region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators), and [GPU pricing](https://cloud.google.com/compute/gpus-pricing).

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
TF = "2.15.1".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

## Get pretrained model from TensorFlow Hub

For demonstration purposes, this tutorial uses a pretrained model from TensorFlow Hub (TFHub), which is then uploaded to a Vertex AI model resource. Once you have a Vertex AI model resource, the model can be deployed to a Vertex AI endpoint resource.

### Download the pretrained model

First, you download the pretrained model from TensorFlow Hub. The model gets downloaded as a TF.Keras layer. To finalize the model, in this example, you create a `Sequential()` model with the downloaded TFHub model as a layer, and specify the input shape to the model.

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 32, 32, 3])

tfhub_model.summary()

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### Upload the TensorFlow Hub model to a Vertex AI model resource

Finally, you upload the model artifacts from the TFHub model into a Vertex AI model resource using the `upload()` method, with the following parameters:

- `display_name`: A human readable name for the model resource.
- `artifact_uri`: The Cloud Storage location of the model package.
- `serving_container_image_uri`: The serving container image.

Uploading a model to a Vertex AI model resource is a long running operation that may take a few moments 

*Note:* When you upload the model artifacts to a Vertex AI model resource, you specify the corresponding deployment container image.

In [ ]:
model = aiplatform.Model.upload(
    display_name="resnet",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
)

print(model)

## Introduction to custom model evaluations

When training a custom model, you generally perform some form of evaluation of the trained model. Your custom model evaluation can then be imported to the corresponding model in the Vertex AI Model Registry using the `import_model_evaluation()` method. Once imported, you can retreive the custom model evaluation with the `list_model_evaluations()` method. 

The Vertex AI Model Registry supports importing multiple model evaluations for a model where each evaluation is distinquished by a unique `display_name`.


### Create a model evaluation

First, you create a model evaluation in a format that corresponds to one of the predefined schemas for model evaluations. In this example, you use the schema for a classification metric, and specify the following subset of evaluation metrics as a dictionary:

- `logLoss`: The log loss.
- `auPrc`: The accuracy.

You then construct the `ModelEvaluation` object with the following parameters:

- `display_name`: The human readable name for the evaluation metric.
- `metrics_schema_uri`: The schema for the specific type of evaluation metrics.
- `metrics`: The dictionary with the evaluation metrics.

Learn more about [Schemas for evaluation metrics](https://cloud.google.com/vertex-ai/docs/evaluation/introduction#features)

In [ ]:
metrics = {"logLoss": 1.4, "auPrc": 0.85}
print(metrics)

model_eval = gapic.ModelEvaluation(
    display_name="eval",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

### Upload the evaluation metrics to the Model Registry

Next, upload the model's evaluation from the custom training job to the corresponding entry in the Vertex AI Model Registry.

Currently, there isn't yet support for this method in the SDK. Instead, you use the lower level GAPIC API interface.

In [ ]:
API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})

client.import_model_evaluation(parent=model.resource_name, model_evaluation=model_eval)

### List the custom model evaluation

Now that you have uploaded your custom evaluation metric to the corresponding model in the Vertex AI Model Registry, you can retrieve it using the `list_model_evaluations()` method.

In [ ]:
evaluation = model.list_model_evaluations()[0]
print(evaluation.gca_resource)

### Upload a second evaluation metrics to the Model Registry

Next, upload a second model evaluation to the corresponding entry in the Vertex AI Model Registry. In this example, we refer to first evaluation metric as `eval` (from training) and the second as `prod` (from production data).

In [ ]:
metrics = {"logLoss": 1.2, "auPrc": 0.87}
print(metrics)

model_prod = gapic.ModelEvaluation(
    display_name="prod",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

client.import_model_evaluation(parent=model.resource_name, model_evaluation=model_prod)

### List a specific custom model evaluation

Now that you have uploaded your custom evaluation metric to the corresponding model in the Vertex AI Model Registry, you can retrieve a specific evaluation by filtering using `display_name`.

In [ ]:
evaluations = model.list_model_evaluations()
for evaluation in evaluations:
    if evaluation.display_name == "prod":
        print(evaluation.gca_resource)

### Upload version 2 of the TFHub model to the Vertex AI Model Registry

Next, you upload the second version of the TFHub model as a model resource in the Vertex AI Model Registry, with the additional following parameters:

- `parent_model`: The resource name or model ID of an existing model that the newly-uploaded model is a version of. Only set this field when uploading a new version of an existing model.
- `is_default_version`:  When set to `True`, the newly uploaded model version automatically includes the alias "default".
- `version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `version_description`: User description of the model version.

When a subsequent model version is created in the Vertex AI Model Registry, the property `version_id` is automatically incremented. In this example, it is set to 2 (2nd version).

In [ ]:
model_v2 = aiplatform.Model.upload(
    display_name="resnet",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=model.resource_name,
    is_default_version=True,
    version_aliases=["v2"],
    version_description="This is the second version of the model",
)

print(model_v2)

#### Upload an evaluation metrics for version 2 of the model to the Model Registry

Next, upload a model evaluation to the corresponding model version in the Vertex AI Model Registry. *Note*, use `model_v2.resource_name` to refer to version 2 of this model.

In [ ]:
metrics = {"logLoss": 1.0, "auPrc": 0.91}
print(metrics)

model_eval = gapic.ModelEvaluation(
    display_name="eval",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

client.import_model_evaluation(
    parent=model_v2.resource_name, model_evaluation=model_eval
)

### List the evaluations for both versions of the model

Finally, list the number of evaluations for both versions of the model.

In [ ]:
evaluations = model.list_model_evaluations()
print("Model v1 no. of evaluations", len(evaluations))
evaluations = model_v2.list_model_evaluations()
print("Model v2 no. of evaluations", len(evaluations))

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [ ]:
# Delete model resource
model.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False  # set True for deletion
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI